<a href="https://colab.research.google.com/github/minjeeons/private-project/blob/main/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EB%AA%A8%EB%8D%B8%EB%A7%81_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
test=pd.read_csv('/content/gdrive/MyDrive/데이콘 연습/test.csv')
train=pd.read_csv('/content/gdrive/MyDrive/데이콘 연습/train.csv')
sample_submisson=pd.read_csv('/content/gdrive/MyDrive/데이콘 연습/submission.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# SibSp : 동반한 형제, 자매, 배우자 수
# Parch : 동반한 부모, 자식 수
# Cabin : 객실 번호
# Embarked : 승선한 항구명

# 데이터 정제

In [ ]:
# SibSp외 Parch로 새로운 변수 FamilySize 생성하기
train['FamilySize']=train['SibSp']+train['Parch']+1
test['FamilySize']=test['SibSp']+test['Parch']+1

In [ ]:
#혼자 탑승했는지 여부를 알려줄 변수 alone 생성하기
train['isAlone']=0
train.loc[train['FamilySize']==1,'isAlone']=1

test['isAlone']=0
test.loc[test['FamilySize']==1,'isAlone']=1

In [ ]:
#Embarked의 결측치를 빈도가 가장 많은 'S'로 대체하기
train['Embarked'].value_counts(ascending=False)
train['Embarked']=train['Embarked'].fillna('S')
test['Embarked']=test['Embarked'].fillna('S')

In [ ]:
#Fare의 결측치를 중위값으로 대체하기
train['Fare']=train['Fare'].fillna(train['Fare'].median())
test['Fare']=test['Fare'].fillna(test['Fare'].median())

In [ ]:
#Age는 정규분포를 통하여 결측값을 처리하기
age_avg=train['Age'].mean()
age_std=train['Age'].std()
age_null_count=train['Age'].isnull().sum()
age_null_random_list=np.random.randint(age_avg-age_std,age_avg+age_std,size=age_null_count)
train['Age'][np.isnan(train['Age'])]=age_null_random_list #데이터가 nan인지 판별
train['Age']=train['Age'].astype(int)

age_avg=test['Age'].mean()
age_std=test['Age'].std()
age_null_count=test['Age'].isnull().sum()
age_null_random_list=np.random.randint(age_avg-age_std,age_avg+age_std,size=age_null_count)
test['Age'][np.isnan(test['Age'])]=age_null_random_list #데이터가 nan인지 판별
test['Age']=test['Age'].astype(int)


<ipython-input-15-389f9e7e0eda>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Age'][np.isnan(train['Age'])]=age_null_random_list #데이터가 nan인지 판별
<ipython-input-15-389f9e7e0eda>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Age'][np.isnan(test['Age'])]=age_null_random_list #데이터가 nan인지 판별


In [ ]:
#Name에서 호칭 추출하기
train['Name']=train['Name'].str.extract('([A-Za-z]+)\.',expand=False)
test['Name']=test['Name'].str.extract('([A-Za-z]+)\.',expand=False)

#적은 수의 호칭은 'Rare'로 대체합
train['Name']=train['Name'].replace(['Lady','Countess','Capt','Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')
train['Name']=train['Name'].replace('Mlle','Miss')
train['Name'] = train['Name'].replace('Ms', 'Miss')
train['Name'] = train['Name'].replace('Mme', 'Mrs')

test['Name']=test['Name'].replace(['Lady','Countess','Capt','Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')
test['Name']=test['Name'].replace('Mlle','Miss')
test['Name'] = test['Name'].replace('Ms', 'Miss')
test['Name'] = test['Name'].replace('Mme', 'Mrs')





In [ ]:
train['Fare'].quantile(0.25)

7.9104

In [ ]:
# 성별을 숫자형으로 변환
train['Sex']=train['Sex'].replace('male',0)
train['Sex']=train['Sex'].replace('female',1)
train['Sex']=train['Sex'].astype(int)

# 호칭을 숫자형으로 변환
name_mapping={'Mr':1,'Miss':2,'Mrs':3,'Master':4,'Rare':5}
train['Name']=train['Name'].map(name_mapping)
train['Name']=train['Name'].fillna(0)

# Embarked를 숫자형으로 변환 .map({'S':0,'C':1,'Q':2})
train['Embarked']=train['Embarked'].replace('S',0)
train['Embarked']=train['Embarked'].replace('C',1)
train['Embarked']=train['Embarked'].replace('Q',2)
train['Embarked']=np.array(train['Embarked'], np.int64)

# Fare를 범주형으로 변환
train.loc[train['Fare']<=7.91,'Fare']=0
train.loc[(train['Fare']>=14.454),'Fare']=1
train.loc[(train['Fare']>=31),'Fare']=2
train.loc[train['Fare']>31,'Fare']=3
train['Fare']=train['Fare'].astype(int)

# Age를 범주형으로 변환
test.loc[test['Age']<=16,'Age']=0
test.loc[(test['Age']>=32),'Age']=1
test.loc[(test['Age']>=48),'Age']=2
test.loc[(test['Age']>=64),'Age']=3
test.loc[test['Age']>64,'Age']=4

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Name        891 non-null    float64
 3   Sex         891 non-null    int64  
 4   Age         891 non-null    int64  
 5   SibSp       891 non-null    int64  
 6   Parch       891 non-null    int64  
 7   Fare        891 non-null    int64  
 8   Embarked    891 non-null    int64  
 9   FamilySize  891 non-null    int64  
 10  isAlone     891 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 76.7 KB


In [ ]:
# 성별을 숫자형으로 변환
test['Sex']=test['Sex'].replace('male',0)
test['Sex']=test['Sex'].replace('female',1)
test['Sex']=test['Sex'].astype(int)

# 호칭을 숫자형으로 변환
name_mapping={'Mr':1,'Miss':2,'Mrs':3,'Master':4,'Rare':5}
test['Name']=test['Name'].map(name_mapping)
test['Name']=test['Name'].fillna(0)

# Embarked를 숫자형으로 변환 .map({'S':0,'C':1,'Q':2})
test['Embarked']=test['Embarked'].replace('S',0)
test['Embarked']=test['Embarked'].replace('C',1)
test['Embarked']=test['Embarked'].replace('Q',2)
test['Embarked']=np.array(test['Embarked'], np.int64)

# Fare를 범주형으로 변환
test.loc[test['Fare']<=7.91,'Fare']=0
test.loc[(test['Fare']>=14.454),'Fare']=1
test.loc[(test['Fare']>=31),'Fare']=2
test.loc[test['Fare']>31,'Fare']=3
test['Fare']=test['Fare'].astype(int)

# Age를 범주형으로 변환
test.loc[test['Age']<=16,'Age']=0
test.loc[(test['Age']>=32),'Age']=1
test.loc[(test['Age']>=48),'Age']=2
test.loc[(test['Age']>=64),'Age']=3
test.loc[test['Age']>64,'Age']=4

In [ ]:
#변수 선별
train=train.drop(['PassengerId','Ticket','Cabin'],axis=1)
test=test.drop(['PassengerId','Ticket','Cabin'],axis=1)

KeyError: ignored

In [ ]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,isAlone
0,0,3,0.0,0,22,1,0,0,0,2,0
1,1,1,0.0,0,38,1,0,0,0,2,0
2,1,3,0.0,0,26,0,0,0,0,1,1
3,1,1,0.0,0,35,1,0,0,0,2,0
4,0,3,0.0,0,35,0,0,8,0,1,1


In [ ]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,isAlone
0,3,0.0,0,0,0,0,0,2,1,1
1,3,0.0,1,0,1,0,0,0,2,0
2,2,0.0,0,0,0,0,9,2,1,1
3,3,0.0,0,27,0,0,8,0,1,1
4,3,0.0,1,22,1,1,12,0,3,0


# 모델 훈련을 위해 데이터 조정

In [ ]:
train_x=train.drop(['Survived'],axis=1)
train_y=train['Survived']
test_x=test

In [ ]:
train_x.shape,train_y.shape,test_x.shape

((891, 10), (891,), (418, 10))

# 모델 훈련

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(train_x,train_y)
Y_pred=logreg.predict(test_x)

In [ ]:
sample_submisson['Survived']=Y_pred

In [ ]:
sample_submisson.to_csv('submission.csv',index=False)